In [3]:
import gc
import joblib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("../")
import time
import warnings
warnings.simplefilter("ignore")
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [4]:
from utils.common import sigmoid, reverse_sigmoid
from utils.constants import *
from utils.extraction_helpers import read_file

In [5]:
%load_ext autoreload
%autoreload

### Gambling6

In [30]:
chris  = read_file(f"{SUBMISSION_DATA_PATH}/sorted_chris.csv")
waikey = read_file(f"{SUBMISSION_DATA_PATH}/wk_seed_6666.csv")
brute_force = read_file(f"{SUBMISSION_DATA_PATH}/1.others_brute_force.csv")
ensemble = read_file(f"{SUBMISSION_DATA_PATH}/2.others_ensemble.csv")
exp_ensemble = read_file(f"{SUBMISSION_DATA_PATH}/3.others_exponential_ensemble.csv")
blend_trees = read_file(f"{SUBMISSION_DATA_PATH}/4.others_blend_boosted_tree.csv")

Shape of data: (924621, 2)
Shape of data: (924621, 2)
Shape of data: (924621, 2)
Shape of data: (924621, 2)
Shape of data: (924621, 2)
Shape of data: (924621, 2)


In [22]:
waikey["sigmoid_prediction"] = waikey["prediction"].apply(sigmoid)

In [23]:
pre_blend_df = pd.DataFrame(
    dict(
        wk=waikey["sigmoid_prediction"],
        chris=chris["prediction"],
        ensemble=ensemble["prediction"],
        exp_ensemble=exp_ensemble["prediction"],
        blend_trees=blend_trees["prediction"]
    )
)

In [24]:
mscaler = MinMaxScaler()
s1 = mscaler.fit_transform(pre_blend_df[["ensemble"]])[:, 0]
s2 = mscaler.fit_transform(pre_blend_df[["blend_trees"]])[:, 0]

In [25]:
weight = 0.6
pre_blend_df["tree_ensemble"] = weight * s1 + (1 - weight) * s2
pre_blend_df = pre_blend_df.drop(columns=["blend_trees", "ensemble"], errors="ignore")

In [26]:
mscaler = MinMaxScaler()
pre_blend_df["exp_ensemble"] = mscaler.fit_transform(pre_blend_df[["exp_ensemble"]])[:, 0]
# pre_blend_df["chris"] = mscaler.fit_transform(pre_blend_df[["chris"]])[:, 0]
# pre_blend_df["wk"] = mscaler.fit_transform(pre_blend_df[["wk"]])[:, 0]

In [18]:
pre_blend_df.corr()

,wk,chris,exp_ensemble,tree_ensemble,final1
wk,1.000000,0.975540,0.984327,0.996924,0.998123
chris,0.975540,1.000000,0.979640,0.974753,0.986200
exp_ensemble,0.984327,0.979640,1.000000,0.989383,0.990883
tree_ensemble,0.996924,0.974753,0.989383,1.000000,0.997000
final1,0.998123,0.986200,0.990883,0.997000,1.000000


In [28]:
pre_blend_df["final1"] = (
    0.59 * pre_blend_df["wk"] + 0.19 * pre_blend_df["chris"] + 0.09 * pre_blend_df["tree_ensemble"] + 0.13 * pre_blend_df["exp_ensemble"]
)

In [29]:
pre_blend_df.describe()

,wk,chris,exp_ensemble,tree_ensemble,final1
count,924621.000000,924621.000000,924621.000000,924621.000000,924621.000000
mean,0.258294,0.310571,0.327446,0.262652,0.277609
std,0.355389,0.384566,0.323079,0.337326,0.353231
min,0.000029,0.000488,0.000000,0.000044,0.000253
25%,0.002838,0.003902,0.067369,0.017686,0.012903
50%,0.027534,0.049972,0.195886,0.057532,0.055014
75%,0.517931,0.712536,0.577492,0.493377,0.559251
max,0.999887,0.999498,1.000000,0.999606,0.999472


In [20]:
pre_blend_df.describe()

,wk,chris,exp_ensemble,tree_ensemble,final1
count,924621.000000,924621.000000,924621.000000,924621.000000,924621.000000
mean,0.258302,0.310391,0.327446,0.262652,0.277579
std,0.355439,0.384948,0.323079,0.337326,0.353332
min,0.000000,0.000000,0.000000,0.000044,0.000143
25%,0.002809,0.003417,0.067369,0.017686,0.012794
50%,0.027509,0.049533,0.195886,0.057532,0.054915
75%,0.517976,0.712754,0.577492,0.493377,0.559319
max,1.000000,1.000000,1.000000,0.999606,0.999634


In [34]:
f = pd.read_csv(f"{SUBMISSION_DATA_PATH}/gambling6_0.801.csv")
p = f["prediction"]

In [35]:
(pre_blend_df["final1"] - p).describe()

count    924621.000000
mean         -0.000033
std           0.000034
min          -0.000146
25%          -0.000054
50%          -0.000034
75%          -0.000013
max           0.000082
dtype: float64

In [60]:
# sub = pd.read_csv(f"{SUBMISSION_DATA_PATH}/empty.csv")
# sub["prediction"] = pre_blend_df["final1"].values
# sub.to_csv(f"{SUBMISSION_DATA_PATH}/gambling6.csv", index=False)